In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Heart Disease/heart.csv')

In [3]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# TRAIN MODEL

In [4]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=120)
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

In [5]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9922178988326849

In [8]:
import pickle
with open('./models/heart_disease_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

# LOAD MODEL

In [10]:
import pickle
with open('./models/heart_disease_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())

21

# LIME

In [11]:
import lime
import lime.lime_tabular
import tqdm 

In [16]:
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['No Disease', 'Disease'], categorical_features=categorical_features, discretize_continuous=True)

In [17]:
text_x = X_test.values

In [18]:
exp_fn = lambda i: explainer.explain_instance(X_test.iloc[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [19]:
import metrics

In [20]:
exp1 = exp_fn_wrap(text_x)
exp2 = exp_fn_wrap(text_x)

100%|██████████| 257/257 [29:48<00:00,  6.96s/it]


In [23]:
with open('./explanations/heart_disease1.pkl', 'wb') as f:
    pickle.dump(exp1, f)
    
with open('./explanations/heart_disease2.pkl', 'wb') as f:
    pickle.dump(exp2, f)

In [24]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [30]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [31]:
i, s, sb

((100.0, 0, 257), (0, 257, 66049, 0.0), (66, 257))

In [33]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [34]:
sim

0.29290093274366613

In [38]:
trust = metrics.calc_trust_score(automl, text_x, exp1, 3, X_train.columns.tolist())

In [ ]:
trust